In [8]:
import sys; print(sys.version)
import os
import glob
import subprocess
import multiprocessing
import numpy as np; print('numpy', np.__version__)
import pandas as pd; print('pandas',pd.__version__)
import allel; print('allel', allel.__version__)
import zarr; print('zarr', zarr.__version__)

from IPython.display import display, HTML


3.7.8 | packaged by conda-forge | (default, Jul 31 2020, 02:25:08) 
[GCC 7.5.0]
numpy 1.19.4
pandas 1.1.4
allel 1.3.2
zarr 2.6.1


In [12]:
INFN= '/home/mdelima/share/aedes/aedes-l4-california.vcf.gz'

OUTFN = INFN+'.zarr'

# Below will probably not need to be changed
FIELDS = [
    'samples',
    'variants/CHROM',
    'variants/POS',
    'variants/REF',
    'variants/ALT',
    'variants/QUAL',
    'variants/TYPE',
#     'variants/NUMALT',
    'variants/numalt',
    'variants/AN',
    'variants/AC',
    'variants/AF',
    'variants/DP',
#     'variants/ANN',
    'calldata/DP',
    'calldata/GT',
         ]
EXCLUDE_FIELDS = None

# # ALTERNATIVE:
# # All the fields from the VCF (overkill leads to very big archive)
# FIELDS = '*' 
# EXCLUDE_FIELDS = ['variants/NUMALT'] # allel will calculate a numalt (lower case) on the fly

TABIX_EXEC = 'tabix'



In [13]:
# get list of chroms
chroms = subprocess.check_output([TABIX_EXEC,'-l',INFN], 
                                 universal_newlines=True).strip().split('\n')
display(chroms)

['1', '2', '3']

In [14]:
# Create zarr archive with a group for each chrom
num_procs = multiprocessing.cpu_count()-1

transformers = None
if 'ANN' in FIELDS:
    transformers=allel.ANNTransformer()

def vcf_to_zarr_func(ch):
    allel.vcf_to_zarr(INFN, OUTFN,
                      region=ch,
                      group=ch,
                      log=sys.stderr,
                      fields=FIELDS,
                      exclude_fields=EXCLUDE_FIELDS,
                      tabix=TABIX_EXEC,
                      transformers=transformers)
    
with multiprocessing.Pool(num_procs) as pool:
    pool.map(vcf_to_zarr_func, chroms, chunksize=1)

[vcf_to_zarr] 65536 rows in 6.69s; chunk in 6.69s (9796 rows/s); 3 :10932448
[vcf_to_zarr] 65536 rows in 6.95s; chunk in 6.95s (9430 rows/s); 2 :8266201
[vcf_to_zarr] 65536 rows in 7.02s; chunk in 7.02s (9339 rows/s); 1 :14470112
[vcf_to_zarr] 131072 rows in 14.13s; chunk in 7.44s (8807 rows/s); 3 :18309102
[vcf_to_zarr] 131072 rows in 14.53s; chunk in 7.58s (8645 rows/s); 2 :16078684
[vcf_to_zarr] 131072 rows in 14.57s; chunk in 7.55s (8677 rows/s); 1 :27259471
[vcf_to_zarr] 196608 rows in 21.23s; chunk in 7.10s (9229 rows/s); 3 :27725274
[vcf_to_zarr] 196608 rows in 21.73s; chunk in 7.16s (9150 rows/s); 1 :39125791
[vcf_to_zarr] 196608 rows in 21.74s; chunk in 7.21s (9090 rows/s); 2 :23205368
[vcf_to_zarr] 262144 rows in 28.28s; chunk in 7.05s (9297 rows/s); 3 :37866922
[vcf_to_zarr] 262144 rows in 28.59s; chunk in 6.85s (9565 rows/s); 2 :31200075
[vcf_to_zarr] 262144 rows in 28.79s; chunk in 7.06s (9281 rows/s); 1 :51378029
[vcf_to_zarr] 327680 rows in 35.34s; chunk in 7.06s (9280 r

In [15]:
callset = zarr.open_group(OUTFN, mode='r')

In [16]:
%%time
callset['2'].tree()

CPU times: user 3.04 ms, sys: 0 ns, total: 3.04 ms
Wall time: 2.93 ms


Tree(nodes=(Node(disabled=True, name='2', nodes=(Node(disabled=True, name='calldata', nodes=(Node(disabled=Tru…

In [17]:
t = callset
list(callset['2/variants'].keys())

['AC',
 'AF',
 'ALT',
 'AN',
 'CHROM',
 'DP',
 'POS',
 'QUAL',
 'REF',
 'TYPE',
 'numalt']

In [18]:
callset

<zarr.hierarchy.Group '/' read-only>